# Timeseries Forecasting on JENA Climate Dataset in R

In [25]:
install.packages("keras3")

also installing the dependency ‘dotty’





The downloaded binary packages are in
	/var/folders/lr/dnbpm8g94tbb2hqq4ksn6lgr0000gn/T//RtmpKEuyZz/downloaded_packages


In [26]:
library("keras3")

Registered S3 methods overwritten by 'keras3':
  method                               from 
  as.data.frame.keras_training_history keras
  plot.keras_training_history          keras
  print.keras_training_history         keras
  r_to_py.R6ClassGenerator             keras


Attaching package: ‘keras3’


The following object is masked _by_ ‘.GlobalEnv’:

    normalize


The following objects are masked from ‘package:keras’:

    %<-active%, %py_class%, activation_elu, activation_exponential,
    activation_gelu, activation_hard_sigmoid, activation_linear,
    activation_relu, activation_selu, activation_sigmoid,
    activation_softmax, activation_softplus, activation_softsign,
    activation_tanh, adapt, application_densenet121,
    application_densenet169, application_densenet201,
    application_efficientnet_b0, application_efficientnet_b1,
    application_efficientnet_b2, application_efficientnet_b3,
    application_efficientnet_b4, application_efficientnet_b5,
    application_efficie

## Get the Processed Dataset

In [17]:
df <- read.csv("jena_climate_2009_2016.csv", check.names = FALSE)
# Check the actual column names
colnames(df)
head(df)

[1] "Date Time"       "p (mbar)"        "T (degC)"        "Tpot (K)"       
 [5] "Tdew (degC)"     "rh (%)"          "VPmax (mbar)"    "VPact (mbar)"   
 [9] "VPdef (mbar)"    "sh (g/kg)"       "H2OC (mmol/mol)" "rho (g/m**3)"   
[13] "wv (m/s)"        "max. wv (m/s)"   "wd (deg)"

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
3,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
4,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
5,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
6,01.01.2009 01:00:00,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7


In [18]:
step <- 6
past <- 720
future <- 72
learning_rate <- 0.001
batch_size <- 256
epochs <- 20

In [19]:
split_fraction <- 0.715
train_split <- floor(split_fraction * nrow(df))

In [20]:
start <- past + future
end <- start + train_split

In [30]:
train_data <- df[1:train_split-1, ]
val_data <- df[train_split:nrow(df), ]

In [31]:
# Select first 7 columns from train_data (columns 1-7 in R)
x_train <- as.matrix(train_data[, 1:7])

# Select rows from start to end, and column 2 (index 1 in Python, but R is 1-indexed)
# Note: Python's iloc[start:end] is exclusive of end, so in R we use (start+1):end
y_train <- df[start:end-1, 2]

# Calculate sequence length
sequence_length <- as.integer(past / step)

In [33]:
dataset_train = keras3::timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length = sequence_length,
    sampling_rate = step,
    batch_size = batch_size,
)

Done!



In [36]:
# x_end should be number of rows, not columns
x_end <- nrow(val_data) - past - future
label_start <- train_split + past + future

In [37]:
# Select rows 1 to x_end, and first 7 columns from val_data
x_val <- as.matrix(val_data[1:x_end, 1:7])

# Select rows from label_start to end, and column 2 (index 1 in Python)
y_val <- df[label_start:nrow(df), 2]

In [38]:
dataset_val = keras3::timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length = sequence_length,
    sampling_rate = step,
    batch_size = batch_size,
)

In [40]:
# Get the first batch from the dataset
# In R/keras3, we use as_iterator() and iter_next()
iter <- as_iterator(dataset_train)
batch <- iter_next(iter)
inputs <- batch[[1]]
targets <- batch[[2]]

# Print shapes (dimensions in R)
cat("Input shape:", paste(dim(inputs), collapse = " x "), "\n")
cat("Target shape:", paste(dim(targets), collapse = " x "), "\n")

Input shape: 256 x 120 x 7 
Target shape: 256 
